# HW4 Review of Python Classes

(also see: https://realpython.com/python3-object-oriented-programming/#how-to-define-a-class-in-python)

The following is meant to be a brief review of classes in Python. If you feel comfortable with building and manipulating classes in Python, feel free to skim this part

## Other python tricks that will be touched upon in our review

* using decorators (see https://realpython.com/primer-on-python-decorators/)
* built-in "magic" methods for classes (see https://rszalski.github.io/magicmethods/)

## Other things not mentioned, but that are also useful to know
* lambda functions (see https://realpython.com/python-lambda/)
* abstract base classes (see https://pymotw.com/2/abc/)
* async (https://realpython.com/async-io-python/)
* multiple inheritance/mixin classes (see https://realpython.com/inheritance-composition-python/)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 1a) Create a class "`Dog`" with an `__init__` function that takes in the variable `name`

In [2]:
class Dog:

    def __init__(self, name):
        self.name = name
        
mydog = Dog('Jojo')
print(mydog.name)

Jojo


### 1b) Add a method to this class `add_trick` that takes in a variable `trick`
Keep in mind that a Dog might learn many tricks - you might keep track of the tricks it learns in a list

In [3]:
class Dog:

    def __init__(self, name): 
        self.name = name
        self.tricks = []
        
    def add_trick(self, trick):
        self.tricks.append(trick)
        
# test your code
mydog = Dog('Jojo')
mydog.add_trick('roll over')
mydog.add_trick('play dead')
mydog.add_trick('flippy flip')

print(mydog.tricks)

['roll over', 'play dead', 'flippy flip']


### 1c) Modify the class so that it keeps track of the dog's age, gender and species
Would these be better as a data attributes or as a method attributes?

In [4]:
class Dog:
    
    species = 'mammal'

    def __init__(self, name, age, gender): 
        
        # example of checking types for arguments passed
        # do not use type(name) == str, 
        # but the isinstance method works with inheritance
        assert isinstance(name, str)
        assert isinstance(age, (int, float))
        assert gender in ('m', 'f', 'o')
        
        self.name = name
        self.age = age
        self.gender = gender
        self.tricks = []
        
    def add_trick(self, trick):
        self.tricks.append(trick)
        
    def update_age(self, age):
        self.age = age
        
mydog = Dog('Jojo', 5, 'm')
# just so you can see that species is assigned at the class level
print(Dog.species)
print(mydog.species)
# update age
mydog.update_age(6)
# however one can also update the attribute age like this
mydog.age = 6

mammal
mammal


### <font color=crimson>Achtung!</font>
Attributes can be directly reassigned for a class! Sometimes we want to prevent this behavior, since reassignment will not go through the checks we wrote in the init. E.g. this will work with our class:

In [5]:
mydog.gender = 'blub'
print('gender: ', mydog.gender)
# this prints the attributes the class contains in a dictionary format
print(mydog.__dict__)

gender:  blub
{'name': 'Jojo', 'age': 6, 'gender': 'blub', 'tricks': []}


# <font color=green>1. Umweg</font>: Decorators

* What are decorators?
* How can I use them?
* Give me examples!

## What are decorators?

A decorator takes in a function (it can also be a class or a method of a class) and adds some functionality and returns the function. Think of it as a wrapper around your function.

This is also called metaprogramming as a part of the program tries to modify another part of the program at compile time.

## How can I use decorators?

There are various built-in decorators in Python such as, property, staticmethod, classmethod and the module functools has some decorators as well. You can also built your own decorators.

<font size=1>NB: If you are using Python 3.8, you can also use a decorator called dataclass (it's pretty cool; take a look: https://docs.python.org/3/library/dataclasses.html). For the class, we are making all code Python 3.6 compatible, so we will not be showing how to use the dataclass decorator. </font>

## Give me examples!

In [6]:
def smart_divide(func):
    # function will accept any arguments
    # *args is a tuple of positional arguments
    # **kwargs is a dictionary of keyword arguments
    def inner(*args, **kwargs):
        try:
            output = func(*args, **kwargs)
            print("No division error!")
            return output
        # catch ZeroDivisionError
        except ZeroDivisionError as e:
            print("Whoops! cannot divide")
            return
    return inner

@smart_divide
def divide(a, b):
    return a/b

def dumb_divide(a, b):
    return a/b

try:
    print(dumb_divide(1, 0))
except ZeroDivisionError as e:
    print('An error was raised: ', e)
# error is caught in decorator
print(divide(1, 0))

print('Examples with no division error:')
# no change when there is no error
print(dumb_divide(2, 1))
print(divide(2, 1))

An error was raised:  division by zero
Whoops! cannot divide
None
Examples with no division error:
2.0
No division error!
2.0


## Examples of using decorators within a class (built-in)

In [7]:
import pickle  # a module to serialize any python object
# more: https://www.geeksforgeeks.org/understanding-python-pickling-example/

# these are examples of built-in decorators in Python
# here is some documentation on built-in function in Python:
# https://docs.python.org/3/library/functions.html

class Test:
    
    def __init__(self, a, b):
        self._a = a
        self._b = b
        
    @property
    def a(self):
        return self._a
    
    @property
    def b(self):
        return 'b={}'.format(self._b)
    
    def a_not_decorated(self):
        return self._a
    
    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.__dict__, f)
    
    @classmethod
    def load(cls, filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        return cls(a=data['_a'], b=data['_b'])
    
    def load_not_class(self, filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        # reinitializing here! this doesn't make sense
        return self.__class__(a=data['_a'], b=data['_b'])

In [8]:
test = Test(a=1, b=2)

print(test.a)
print(test.b)
print(test.a_not_decorated)
print(test.a_not_decorated())
test.save('test.txt')

print('loaded instance')
test_loaded = Test.load('test.txt')
# this still works but i initialize the class 
# and then after loading i initialize it anew
test_loaded2 = Test(a=1, b=2).load_not_class('test.txt')
print(test_loaded.a)
print(test_loaded.b)

1
b=2
<bound method Test.a_not_decorated of <__main__.Test object at 0x11a9c15d0>>
1
loaded instance
1
b=2


In [9]:
test.a = 'new object'

AttributeError: can't set attribute

In [10]:
# setter decorators

class Test:
    
    def __init__(self, a, b):
        assert isinstance(a, (int, float)), 'must be float or int'
        self._a = a
        self._b = b
        
    @property
    def a(self):
        return self._a
    
    @a.setter
    def a(self, value):
        assert isinstance(value, (int, float)), 'must be float or int'
        self._a = value
    
    @property
    def b(self):
        return 'b={}'.format(self._b)
    
    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.__dict__, f)
    
    @classmethod
    def load(cls, filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        return cls(a=data['_a'], b=data['_b'])

In [11]:
test2 = Test(1, 'asdf')

print(test2.a)
print(test2.b)

test2.a = 'asdf'

1
b=asdf


AssertionError: must be float or int

In [12]:
test2.a = 4

# now a has a new value
print(test2.a)

4


## Incorporating decorators into our dog class

In [13]:
class Dog:
    
    species = 'mammal'

    def __init__(self, name, age, gender): 
        
        # example of checking types for arguments passed
        # do not use type(name) == str, 
        # but the isinstance method works with inheritance
        assert isinstance(name, str)
        assert isinstance(age, (int, float))
        assert gender in ('m', 'f', 'o')
        
        self._name = name
        self._age = age
        self._gender = gender
        self._tricks = []
        
    @property
    def name(self):
        return self._name
    
    @property
    def age(self):
        return self._age
    
    @property
    def gender(self):
        return self._gender
    
    @property
    def tricks(self):
        return self._tricks
        
    def add_trick(self, trick):
        self.tricks.append(trick)
        
    def update_age(self, age):
        self._age = age

In [14]:
mydog = Dog('Jojo', 5, 'm')
# update age
mydog.update_age(6)
print(mydog.age)
# you cannot update by just reassigning
mydog.age = 6

6


AttributeError: can't set attribute

# RETURN TO HOMEWORK

### 1d) Create 3 dogs with various characteristics, and assign them different tricks

In [15]:
dog1 = Dog('Luke', 5, 'm')
dog1.add_trick('swing a lightsaber')
dog1.add_trick('bark "NOOOOOOOO"')
dog2 = Dog('Leia', 5, 'f')
dog2.add_trick('lead a rebellion')
dog2.add_trick('acquire the best diss track')
dog3 = Dog('Han', 6, 'm')
dog3.add_trick('shoot first')

### 1e) Create a method/function `print_summary` for the Dog class that prints all the relevant data and tricks

In [16]:
class Dog:
    
    species = 'mammal'

    def __init__(self, name, age, gender): 
        
        # example of checking types for arguments passed
        # do not use type(name) == str, 
        # but the isinstance method works with inheritance
        assert isinstance(name, str)
        assert isinstance(age, (int, float))
        assert gender in ('m', 'f', 'o')
        
        self.name = name
        self.age = age
        self.gender = gender
        self.tricks = []
        
    def add_trick(self, trick):
        self.tricks.append(trick)
        
    def update_age(self, age):
        self.age = age
        
    def print_summary(self):
        # select pronoun
        pronoun = {
            'm':'he', 
            'f':'she', 
        }.get(self.gender, 'they')
        # create tricks string
        tricks = ' and '.join(self.tricks)
        # print
        print((
            "{0} is a good dog. At "
            "the young age of {1}, {2} can {3}."
        ).format(self.name, self.age, pronoun, tricks))

In [17]:
dog1 = Dog('Luke', 5, 'm')
dog1.add_trick('swing a lightsaber')
dog1.add_trick('bark "NOOOOOOOO"')
dog2 = Dog('Leia', 5, 'f')
dog2.add_trick('lead a rebellion')
dog2.add_trick('acquire the best diss track')
dog3 = Dog('Han', 6, 'm')
dog3.add_trick('shoot first')
dogs = [dog1, dog2, dog3]

for d in dogs:
    d.print_summary()

Luke is a good dog. At the young age of 5, he can swing a lightsaber and bark "NOOOOOOOO".
Leia is a good dog. At the young age of 5, she can lead a rebellion and acquire the best diss track.
Han is a good dog. At the young age of 6, he can shoot first.


# <font color=green>2. Umweg</font>: "Magic" methods

* What are built-in methods?
* Why do I want to use them?
* Give me some examples!

## What are built-in methods?

What are magic methods? They're everything in object-oriented Python. They're special methods that you can define to add "magic" to your classes. They're always surrounded by double underscores (e.g. `__init__` or `__lt__`).

Each magic method has defined way of working within the Python interpreter. For example, the `__init__` is used during initialization. 

There many ways that a "magic" method has already been applied without you noticing.

## Why do I want to use magic?

Let's take an example of a magic method in use:
```python
a = YourClass('a')
b = YourClass('b')
```
Let's say you want to compare your two instances `a` and `b` to see if they equal each other. That is you want to use `==` as you would do for an native Python object:
```
float1 = 1.0
float2 = 2.0
float1 == float2 # this will return False
```

The `==` symbol is implemented by the magic `__eq__` method. 
So in `YourClass` you would need to implement a method called `__eq__` that accepts one other argument besides self, which is the other objects it is being compared to:
```python
class YourClass:
    
    def __init__(self, name):
        self.name = name
    
    def __eq__(self, other):
        return (
            isinstance(other, self.__class__) 
            and (self.name == other.name)
        )
```

Basically you can use the magic methods to define how your class should act when you for example use the `+`, `-`, or other built-in symbols or when you use built-in functions like `print`. This can give your class some operations that may make your code more efficient and/or easier to read. 

## Give me examples!

In [18]:
class YourClass:

    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return (
            isinstance(other, self.__class__) 
            and (self.name == other.name)
        )

In [19]:
dir(YourClass)
# some magic methods have a default implementation

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [20]:
a = YourClass('a')
b = YourClass('b')
c = YourClass('a')

print(a == b)
print(a == c)
print(a == 1)
print(a == 'a')

False
True
False
False


#### Let's add a string method instead of a print_summary method to our Dog class

In [21]:
class Dog:
    
    species = 'mammal'

    def __init__(self, name, age, gender): 
        
        # example of checking types for arguments passed
        # do not use type(name) == str, 
        # but the isinstance method works with inheritance
        assert isinstance(name, str)
        assert isinstance(age, (int, float))
        assert gender in ('m', 'f', 'o')
        
        self.name = name
        self.age = age
        self.gender = gender
        self.tricks = []
        
    def add_trick(self, trick):
        self.tricks.append(trick)
        
    def update_age(self, age):
        self.age = age
        
    def __str__(self):
        # select pronoun
        pronoun = {
            'm':'he', 
            'f':'she', 
        }.get(self.gender, 'they')
        # create tricks string
        tricks = ' and '.join(self.tricks)
        # return string
        return (
            "{0} is a good boy. At "
            "the young age of {1}, {2} can {3}."
        ).format(self.name, self.age, pronoun, tricks)

In [22]:
dog1 = Dog('Luke', 5, 'm')
dog1.add_trick('swing a lightsaber')
dog1.add_trick('bark "NOOOOOOOO"')
print(dog1)

Luke is a good boy. At the young age of 5, he can swing a lightsaber and bark "NOOOOOOOO".


## More advanced example: Let's build a polynomial class
Example from: https://www.python-course.eu/polynomial_class_in_python.php

$P(x) = a_{n}x^{n}+a_{n-1}x^{n-1}+\dotsb +a_{2}x^{2}+a_{1}x+a_{0}$

![alt text](static/polynomials.png)

In [23]:
# this is a built-in package in python
from itertools import zip_longest

# do not focus too much what each method is doing
# but that you can define these magic functions and use
# +, -, (), and len because of it
class Polynomial:
    
    def __init__(self, *coefficients):
        """ 
        coefficients are in the form a_n, ...a_1, a_0 
        """
        self.coefficients = list(coefficients) # tuple is turned into a list
            
    def __call__(self, x):    
        res = 0
        for coeff in self.coefficients:
            res = res * x + coeff
        return res 
    
    def __str__(self):
        """string representation of polynomial"""
        return self.__class__.__name__ + str(self.coefficients)
    
    def __len__(self):
        return len(self.coefficients) - 1  
            
    def __add__(self, other):
        c1 = self.coefficients[::-1]
        c2 = other.coefficients[::-1]
        res = [sum(t) for t in zip_longest(c1, c2, fillvalue=0)]
        return self.__class__(*res[::-1])
    
    def __sub__(self, other):
        c1 = self.coefficients[::-1]
        c2 = other.coefficients[::-1]
        
        res = [t1-t2 for t1, t2 in zip_longest(c1, c2, fillvalue=0)]
        return self.__class__(*res[::-1])

In [24]:
pol1 = Polynomial(1, 2, 3)
pol2 = Polynomial(2, 3)

print('nicely printing polynomials')
print(pol1)
print(pol2)
print('\n') # these are just line break

print('length of polynomials')
print(len(pol1))
print(len(pol2))
print('\n')

print('call on polynomials to evaluates for a particular x')
print(pol1(1))
print(pol1(np.arange(100)))
print('\n')

print('adding and subtracting polynomials')
pol3 = pol1 + pol2
pol4 = pol3 - pol2
print(pol3)
print(pol4)

nicely printing polynomials
Polynomial[1, 2, 3]
Polynomial[2, 3]


length of polynomials
2
1


call on polynomials to evaluates for a particular x
6
[    3     6    11    18    27    38    51    66    83   102   123   146
   171   198   227   258   291   326   363   402   443   486   531   578
   627   678   731   786   843   902   963  1026  1091  1158  1227  1298
  1371  1446  1523  1602  1683  1766  1851  1938  2027  2118  2211  2306
  2403  2502  2603  2706  2811  2918  3027  3138  3251  3366  3483  3602
  3723  3846  3971  4098  4227  4358  4491  4626  4763  4902  5043  5186
  5331  5478  5627  5778  5931  6086  6243  6402  6563  6726  6891  7058
  7227  7398  7571  7746  7923  8102  8283  8466  8651  8838  9027  9218
  9411  9606  9803 10002]


adding and subtracting polynomials
Polynomial[1, 4, 6]
Polynomial[1, 2, 3]


Take a look at all the magic functions that exist: https://rszalski.github.io/magicmethods/ !!!

You can basically customize every aspect of how your class works, even how it should get attributes in the class or you can define how it should index or set an index.

## Other things not mentioned, but that are also useful to know
* lambda functions (see https://realpython.com/python-lambda/)
* abstract base classes (see https://pymotw.com/2/abc/)
* async (https://realpython.com/async-io-python/)
* multiple inheritance/mixin classes (see https://realpython.com/inheritance-composition-python/)

# E.g. Remember inheritance 

With inheritance, you can inherit all the methods from the parent class and write new or overwrite old methods. 

In [25]:
class RussellTerrier(Dog):
    def run(self, speed):
        return "{} runs at a speed of {}km/h".format(self.name, speed)
    
class SuperRussellTerrier(RussellTerrier):
    def run(self, speed):
        return super().run(speed * 10)

class Bulldog(Dog):
    def run(self, speed):
        return "{} begrudgingly runs at a speed of {}km/h".format(self.name, speed)
    
dog1 = RussellTerrier('Barry', age=100, gender='m')
dog2 = SuperRussellTerrier('Flash', age=100, gender='f')
dog3 = Bulldog('Hulk', age=20, gender='m')

print(dog1.run(100))
print(dog2.run(100))
print(dog3.run(3))

Barry runs at a speed of 100km/h
Flash runs at a speed of 1000km/h
Hulk begrudgingly runs at a speed of 3km/h
